# DTGraph Introduction

This notebook consists of simple examples of usage of the library.

## Part 1: Establishing a connection

In [1]:
from dtgraph import Neo4jGraph, Rule

In the following we will connect to a [Neo4j Sandbox](https://sandbox.neo4j.com/).
After having created your instance, you can retrieve the *ip address* and the *password* (e.g., in my case, those are  `54.159.136.111` and `analyzer-information-tails`).

In [2]:
hostname = "54.159.136.111"
password = "analyzer-information-tails"

In [7]:
uri = f"bolt://{hostname}:7687"
graph = Neo4jGraph(uri, "neo4j", username="neo4j", password=password, verbose=True)

You can check that it has some content if you specified the Movie dataset:

In [8]:
graph.output_all_nodes()

The query `
        MATCH (n)
        RETURN COUNT(n) as count
        ` returned 1 records in 46 ms.
Out:    There is currently 171 node(s) in the database.


Then you can write your own transformation:

In [9]:
my_query = Rule.from_ascii('''
        MATCH (n:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(o:Person)
        => 
        (x = (n) : Actor {
            name = n.name,
            born = n.born
        })-[(m) : COLLEAGUE {
            movie = m.title
        }]->(y = (o) : Actor {
            name = o.name,
            born = o.born
        })
''')

And execute it:

In [10]:
my_query.apply_on(graph)

The query `MATCH (n:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(o:Person)
MERGE (x:_dummy {
    _id: "(" + elementID(n) + ")" 
})
ON CREATE
    SET x:Actor,
        x.name = n.name,
        x.born = n.born
ON MATCH
    SET x:Actor,
        x.name = 
        CASE
            WHEN x.name <> n.name THEN
                "Conflict Detected!"
            ELSE
                n.name
        END,
        x.born = 
        CASE
            WHEN x.born <> n.born THEN
                "Conflict Detected!"
            ELSE
                n.born
        END
MERGE (y:_dummy {
    _id: "(" + elementID(o) + ")" 
})
ON CREATE
    SET y:Actor,
        y.name = o.name,
        y.born = o.born
ON MATCH
    SET y:Actor,
        y.name = 
        CASE
            WHEN y.name <> o.name THEN
                "Conflict Detected!"
            ELSE
                o.name
        END,
        y.born = 
        CASE
            WHEN y.born <> o.born THEN
                "Conflict Detected!"
            ELSE
      

2474

Yay, 102 nodes and 204 labels have been created! You can check with the following query on your Neo4j browser the result (alongside the initial dataset, for now):
```
MATCH (n)
RETURN n
```